# Hello!
This is a walkthrough to train the University of Waterloo's submission model at ClimateHack 2023.

To find all our experiments and code, see our original [repo](https://github.com/trevor-yu-087/climatehack.ai-2024), but beware, it is not documented, or well organized.

# Environment set up

We use docker to package dependencies. If you are using VScode or a JetBrains IDE, the devcontainers extension should be able to use the .devcontainer directory to build the docker image and use it as a development environment.

If you do not want to use docker, you can (hopefully) get set up by running:

- `pip install -r local-requirements.txt`
- `conda install cartopy`
- `pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121`

A machine with a CUDA enabled GPU is required.

# Download data
Our model used pv, hrv and weather data.

For this example we'll only be downloading a few months of data.

Note that you have to download the [indices.json](https://github.com/climatehackai/getting-started-2023/blob/main/indices.json) file and place it in the same directory as the data that gets downloaded below.

In [1]:
import huggingface_hub
from os import makedirs

datadir = "/workspaces/waterloo-climatehack/data" # change this
makedirs(datadir, exist_ok=True)

huggingface_hub.snapshot_download(
    repo_id="climatehackai/climatehackai-2023", 
    local_dir=datadir, 
    cache_dir=datadir + '/cache',
    local_dir_use_symlinks=False, 
    repo_type="dataset",
    ignore_patterns=["aerosols/*", "satellite-nonhrv/*"],
    allow_patterns=["*10.zarr.zip", "*11.zarr.zip", "*.parquet", "*metadata.csv"]
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 33 files:   0%|          | 0/33 [00:00<?, ?it/s]

'/workspaces/waterloo-climatehack/data'

# Generating PV  Features
We generate site specific features (such as the site's max and average output during each month).

In [2]:
import pandas as pd
import numpy as np

In [3]:
years = [2020, 2021]
months = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december']

for i, month_name in enumerate(months):
    print(f'Processing {month_name}')
    month = pd.read_parquet([datadir + f'/pv/{year}/{i+1}.parquet' for year in [2020,2021]])
    month = month.drop(['generation_wh'], axis=1).reorder_levels(['ss_id', 'timestamp'])

    site_ids = month.index.get_level_values(0).unique().values

    monthly_avg, monthly_max, monthly_average_max = [], [], []

    for site in site_ids:
        a = month.loc[site].between_time('5:00', '22:00')
        monthly_max.append(a.power.max())
        monthly_avg.append(a.power.mean())
        monthly_average_max.append(a.groupby([a.index.hour, a.index.minute]).power.mean().max())

    frame = pd.DataFrame(np.array([monthly_avg, monthly_max, monthly_average_max]).T, index=site_ids)
    frame.columns = [f'{month_name}_avg', f'{month_name}_max', f'{month_name}_average_max']

    if i == 0:
        pv_metrics_frame = frame
    else:
        pv_metrics_frame = pv_metrics_frame.join(frame)

Processing january
Processing february
Processing march
Processing april
Processing may
Processing june
Processing july
Processing august
Processing september
Processing october
Processing november
Processing december


In [4]:
pv_metrics_frame.head()

,january_avg,january_max,january_average_max,february_avg,february_max,february_average_max,march_avg,march_max,march_average_max,april_avg,...,september_average_max,october_avg,october_max,october_average_max,november_avg,november_max,november_average_max,december_avg,december_max,december_average_max
2607,0.034619,0.726833,0.169660,0.102621,0.919714,0.351934,0.114527,0.978294,0.309300,0.255650,...,0.510344,0.092981,0.912645,0.290926,0.072014,0.807710,0.344670,0.022011,0.553282,0.143006
2626,0.025563,0.347593,0.117526,0.063945,0.510123,0.207965,0.137374,0.813030,0.372940,0.238742,...,0.352374,0.071822,0.683270,0.230175,0.034770,0.498327,0.141312,0.018410,0.219477,0.090823
2631,0.018289,0.214462,0.072340,0.045558,0.415719,0.141130,0.099934,0.623148,0.267659,0.174131,...,0.290042,0.056073,0.513339,0.168292,0.031914,0.296281,0.117800,0.013659,0.177520,0.057554
2657,0.064557,0.814452,0.231149,0.116752,0.913380,0.330586,0.175307,0.937380,0.464801,0.263179,...,0.494600,0.119239,0.915678,0.352494,0.097194,0.833607,0.332203,0.063598,0.818130,0.278857
2660,0.012224,0.374229,0.052229,0.021358,0.492336,0.076837,0.042023,0.637155,0.137369,0.080295,...,0.324225,0.052534,0.587280,0.170331,0.042367,0.396132,0.169838,0.020474,0.268075,0.093186


### Loading PV Metadata

In [5]:
metadata = pd.read_csv(datadir + '/pv/metadata.csv')
metadata.index = metadata.ss_id
metadata.drop(['llsoacd', 'operational_at', 'ss_id'], axis=1, inplace=True)
metadata.head()

,latitude_rounded,longitude_rounded,orientation,tilt,kwp
ss_id,,,,,
2405,53.53,-1.63,180.0,35.0,3.36
2406,54.88,-1.38,315.0,30.0,1.89
2407,54.88,-1.38,225.0,30.0,1.89
2408,54.88,-1.38,225.0,30.0,1.89
2409,54.88,-1.38,225.0,30.0,1.89


### Converting the PV Metrics Dataframe to a Mapping

In [6]:
pv_metric_sites = set(pv_metrics_frame.index)
nan_fill = pv_metrics_frame.mean()

In [7]:
pv_metrics = {}

month_names = ["january", "february", "march", "april", "may", "june", "july", "august", "september", "october", "november", "december"]

for site_id, (lat, lon, orient, tilt, kwp) in metadata[["latitude_rounded", "longitude_rounded", "orientation", "tilt", "kwp"]].iterrows():
    for month_number, month in enumerate(month_names, start=1):
        key = (lat, lon, orient, tilt, kwp)
        metric_names = ["_".join([month, metric]) for metric in ["avg", "max", "average_max"]]
        if site_id not in pv_metric_sites:
            metrics = nan_fill[metric_names].values
        else:
            metrics = pv_metrics_frame.loc[site_id, metric_names].values
        if np.isnan(metrics).any():
            metrics = nan_fill[metric_names].values
        pv_metrics.setdefault(month_number, {})[key] = metrics

In [8]:
import pickle
with open(datadir + "/pv_metrics.pkl", "wb") as f:
    pickle.dump(pv_metrics, f)

# Data Loading
Now let's run our dataset class to validate that our data is set up correctly.

In [9]:
from dataset import get_datasets
import yaml

In [10]:
# yaml file that is used to configure training runs
CONFIG_FILE_NAME = "train.yaml"

with open(CONFIG_FILE_NAME) as file:
    config = yaml.safe_load(file)

In [11]:
train_ds, test_ds = get_datasets(
    config["data_path"],
    (config["start_date"], config["end_date"]),
    batch_size=config["batch_size"],
    hrv="hrv" in config["modalities"],
    weather="weather" in config["modalities"],
    metadata="metadata" in config["modalities"],
    seed=config["seed"],
    pv_features_file=config["pv_features_file"],
    test_size=config["test_size"],
    hrv_crop=config["hrv_crop"],
    weather_crop=config["weather_crop"],
    zipped=config["zipped"],
    offset_start_time=config["offset_start_time"]
)

Loading dataset checking checkpoint


# Set up model training

In [12]:
import torch
from pvlib.solarposition import get_solarposition
from datetime import datetime
from cartopy import crs
from itertools import accumulate
from functools import partial

In [13]:
torch.backends.cuda.matmul.allow_tf32 = True
torch.manual_seed(config["seed"])
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
# Precompile solar position
_ = get_solarposition(
    time=datetime(2020, 1, 2, 3),
    latitude=123.4,
    longitude=49.0,
    method="nrel_numba"
)

/usr/local/lib/python3.10/dist-packages/pvlib/solarposition.py:263: UserWarning: Reloading spa to use numba
  warnings.warn('Reloading spa to use numba')
/usr/local/lib/python3.10/dist-packages/pvlib/spa.py:940: UserWarning: The number of threads is more than the length of the time array. Only using %s threads.
  warnings.warn('The number of threads is more than the length of '


In [15]:
# hrv lat lon features
delta_geos = 1000.1343488693237
ix = np.arange(config["hrv_crop"]) - (config["hrv_crop"] // 2)
xx, yy = np.meshgrid(ix, ix)
xx_hrv = xx * delta_geos
yy_hrv = yy * delta_geos


# weather lat lon features
delta_nwp = 0.0623
ix = np.arange(config["weather_crop"]) - (config["weather_crop"] // 2)
xx, yy = np.meshgrid(ix, ix)
xx_nwp = xx * delta_nwp
yy_nwp = yy * delta_nwp

In [16]:
hrv_coords = crs.Geostationary(central_longitude=9.5, sweep_axis="y")
latlon_coords = crs.Geodetic()   
def get_hrv_lat_lon_features(lat, lon):
    x_geo, y_geo = hrv_coords.transform_point(lon, lat, latlon_coords)

    xx_geo = xx_hrv + x_geo
    yy_geo = yy_hrv + y_geo

    coords = latlon_coords.transform_points(
        hrv_coords,
        xx_geo, yy_geo
    )

    xx_lon = coords[..., 0]
    yy_lat = coords[..., 1]
    features = np.stack((xx_lon, yy_lat), axis=-1)
    return features

In [17]:
def compute_solar_incidence(az, el, orient, tilt):
    # Assume in degrees
    panel_vec = np.array([
        np.cos(np.radians(orient)),
        np.sin(np.radians(orient)),
        np.sin(np.radians(tilt))
    ])

    solar_vec = np.stack([
        np.cos(np.radians(az)),
        np.sin(np.radians(az)),
        np.sin(np.radians(el))
    ], axis=1)

    sim = -solar_vec @ panel_vec.T
    return sim

In [18]:
def worker_init_fn(id, split_seed: int):
    process_seed = torch.initial_seed()
    base_seed = process_seed - id
    ss = np.random.SeedSequence(
        [id, base_seed, split_seed]
    )
    np_rng_seed = ss.generate_state(4)
    np.random.seed(np_rng_seed)

In [19]:
def metadata_collate_fn(batch):
    """Data is already batched
    Weather is already shape (B, C, L, H, W)
    """
    batch = batch[0]
    metadata_features = {}
    for (lat, lon, orient, tilt, kwp), t0 in zip(batch["metadata"], batch["time"]):
        t0 = pd.Timestamp(t0) - pd.Timedelta(hours=1)
        # 60 timestamp including first hour and prediction window
        ts = list(accumulate([pd.Timedelta(minutes=5)] * 59, initial=t0))
        ts = pd.DatetimeIndex(ts)
        solar_pos = get_solarposition(
            time=ts, 
            latitude=lat,
            longitude=lon,
            method="nrel_numba"
        )

        # Scale to [0, 1] for SSP
        doy = ts.day_of_year.values / 365
        mod = ((ts.hour.values * 60) + ts.minute.values) / (24 * 60)
        
        metadata_features.setdefault("time", []).append(np.stack([
            mod,
            doy
        ], axis=1))

        # Weather time features on the hour
        t0 = t0.floor("60min")  # t0 already 1 hr before
        ts = list(accumulate([pd.Timedelta(minutes=60)] * 5, initial=t0))
        ts = pd.DatetimeIndex(ts)
        doy = ts.day_of_year.values / 365
        mod = ((ts.hour.values * 60) + ts.minute.values) / (24 * 60)
        metadata_features.setdefault("weather_time", []).append(np.stack([
            mod,
            doy
        ], axis=1))

        lon_xx = lon + xx_nwp
        lat_yy = lat + yy_nwp
        metadata_features.setdefault("location", []).append(np.stack([
            lon_xx,
            lat_yy
        ], axis=-1))

        metadata_features.setdefault("hrv_location", []).append(get_hrv_lat_lon_features(lat, lon))

        # Scale to [0, 1] for SSP
        az = solar_pos["azimuth"].values / 360
        el = solar_pos["apparent_elevation"].values / 360
        metadata_features.setdefault("azel", []).append(np.stack([
            az,
            el
        ], axis=1))

        # Scale to [0, 1] for SSP
        orient = orient / 360
        tilt = tilt / 360
        metadata_features.setdefault("static", []).append(np.array([
            [orient,
            tilt,
            kwp]
        ]))
        
    batch = {k: torch.FloatTensor(v) for k, v in batch.items() if k not in ["time", "metadata"]}
    for k, v in metadata_features.items():
        batch[k] = torch.FloatTensor(np.stack(v))
    batch["pv"] = batch["pv"].unsqueeze(-1)
    batch["pv_features"] = batch["pv_features"].unsqueeze(-2)
    return batch

In [20]:
train_loader = torch.utils.data.DataLoader(
    train_ds,
    batch_size=1,
    shuffle=False,
    collate_fn=metadata_collate_fn,
    pin_memory=True,
    worker_init_fn=partial(worker_init_fn, split_seed=0),
    num_workers=config["num_workers"]
    )
val_loader = torch.utils.data.DataLoader(
    test_ds, 
    batch_size=1, 
    shuffle=False, 
    collate_fn=metadata_collate_fn, 
    pin_memory=True,
    worker_init_fn=partial(worker_init_fn, split_seed=0),
    num_workers=config["num_workers"]
)